In [3]:
# Cell 1: Install libraries if needed
# !pip install nibabel pandas

# Cell 2: Import libraries
import os
import nibabel as nib
import pandas as pd

# Cell 3: Define your dataset directory
# Adjust this to your dataset directory
data_dir = "/Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/resampled/train/amos_ct/images/"  # Replace with your dataset path

# Cell 4: Collect all NIfTI files
nifti_files = [f for f in os.listdir(data_dir) if f.endswith('.nii') or f.endswith('.nii.gz')]

# Cell 5: Function to inspect shape and spacing
def inspect_nifti(file_path):
    nii = nib.load(file_path)
    shape = nii.shape
    spacing = nii.header.get_zooms()
    affine = nii.affine
    return shape, spacing, affine

# Cell 6: Loop over all files and collect results
results = []

for file in sorted(nifti_files):
    file_path = os.path.join(data_dir, file)
    shape, spacing, affine = inspect_nifti(file_path)
    results.append({
        "File": file,
        "Shape": shape,
        "Spacing (X,Y,Z)": spacing,
        "Affine": affine
    })

# Cell 7: Display results in a DataFrame
df = pd.DataFrame(results)
pd.set_option('display.max_rows', None)  # Show all rows
print(df)

# Optional: Save to CSV
# df.to_csv("slice_shape_spacing_summary.csv", index=False)


                 File            Shape  Spacing (X,Y,Z)  \
0    amos_0001.nii.gz  (192, 192, 192)  (1.0, 1.0, 1.0)   
1    amos_0004.nii.gz  (192, 192, 192)  (1.0, 1.0, 1.0)   
2    amos_0005.nii.gz  (192, 192, 192)  (1.0, 1.0, 1.0)   
3    amos_0006.nii.gz  (192, 192, 192)  (1.0, 1.0, 1.0)   
4    amos_0007.nii.gz  (192, 192, 192)  (1.0, 1.0, 1.0)   
5    amos_0009.nii.gz  (192, 192, 192)  (1.0, 1.0, 1.0)   
6    amos_0010.nii.gz  (192, 192, 192)  (1.0, 1.0, 1.0)   
7    amos_0011.nii.gz  (192, 192, 192)  (1.0, 1.0, 1.0)   
8    amos_0014.nii.gz  (192, 192, 192)  (1.0, 1.0, 1.0)   
9    amos_0015.nii.gz  (192, 192, 192)  (1.0, 1.0, 1.0)   
10   amos_0016.nii.gz  (192, 192, 192)  (1.0, 1.0, 1.0)   
11   amos_0017.nii.gz  (192, 192, 192)  (1.0, 1.0, 1.0)   
12   amos_0019.nii.gz  (192, 192, 192)  (1.0, 1.0, 1.0)   
13   amos_0021.nii.gz  (192, 192, 192)  (1.0, 1.0, 1.0)   
14   amos_0023.nii.gz  (192, 192, 192)  (1.0, 1.0, 1.0)   
15   amos_0024.nii.gz  (192, 192, 192)  (1.0, 1.0, 1.0) 

In [ ]:
# Cell 1: Install SimpleITK if needed
# !pip install SimpleITK

# Cell 2: Import libraries
import os
import SimpleITK as sitk

# Cell 3: Define paths
input_dir = "/Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/amos22_mri/imagesTr/"    # Replace with your AMOS images folder
output_dir = "/Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/amos22_mri_resampled/imagesTr/"   # Replace with your desired output folder
os.makedirs(output_dir, exist_ok=True)

# Cell 4: Define target spacing
# Here we pick (1.0, 1.0, 3.0) as an example; adjust as needed.
target_spacing = (1.0, 1.0, 3.0)  # (X, Y, Z) in mm

# Cell 5: Resampling function
def resample_image(image, target_spacing, is_label=False):
    original_spacing = image.GetSpacing()
    original_size = image.GetSize()
    
    # Calculate new size
    new_size = [
        int(round(original_size[i] * (original_spacing[i] / target_spacing[i])))
        for i in range(3)
    ]
    
    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputSpacing(target_spacing)
    resampler.SetSize(new_size)
    resampler.SetOutputDirection(image.GetDirection())
    resampler.SetOutputOrigin(image.GetOrigin())
    resampler.SetInterpolator(sitk.sitkNearestNeighbor if is_label else sitk.sitkLinear)
    
    resampled_image = resampler.Execute(image)
    return resampled_image

# Cell 6: Process all images
nifti_files = [f for f in os.listdir(input_dir) if f.endswith('.nii') or f.endswith('.nii.gz')]

for file in sorted(nifti_files):
    file_path = os.path.join(input_dir, file)
    image = sitk.ReadImage(file_path)
    resampled = resample_image(image, target_spacing)
    
    output_path = os.path.join(output_dir, file)
    sitk.WriteImage(resampled, output_path)
    
    print(f"Resampled: {file} -> Shape: {resampled.GetSize()}, Spacing: {resampled.GetSpacing()}")

print("✅ Resampling complete!")


Resampled: amos_0507.nii.gz -> Shape: (380, 344, 60), Spacing: (1.0, 1.0, 3.0)
Resampled: amos_0508.nii.gz -> Shape: (366, 120, 150), Spacing: (1.0, 1.0, 3.0)
Resampled: amos_0510.nii.gz -> Shape: (366, 156, 150), Spacing: (1.0, 1.0, 3.0)
Resampled: amos_0514.nii.gz -> Shape: (366, 168, 150), Spacing: (1.0, 1.0, 3.0)
Resampled: amos_0517.nii.gz -> Shape: (400, 400, 77), Spacing: (1.0, 1.0, 3.0)
Resampled: amos_0518.nii.gz -> Shape: (400, 240, 133), Spacing: (1.0, 1.0, 3.0)
Resampled: amos_0522.nii.gz -> Shape: (440, 358, 72), Spacing: (1.0, 1.0, 3.0)
Resampled: amos_0530.nii.gz -> Shape: (380, 344, 64), Spacing: (1.0, 1.0, 3.0)
Resampled: amos_0532.nii.gz -> Shape: (420, 341, 72), Spacing: (1.0, 1.0, 3.0)
Resampled: amos_0538.nii.gz -> Shape: (420, 341, 64), Spacing: (1.0, 1.0, 3.0)
Resampled: amos_0540.nii.gz -> Shape: (375, 375, 67), Spacing: (1.0, 1.0, 3.0)
Resampled: amos_0541.nii.gz -> Shape: (380, 344, 72), Spacing: (1.0, 1.0, 3.0)
Resampled: amos_0548.nii.gz -> Shape: (380, 309,

In [ ]:
# -*- coding: utf-8 -*-
"""
NIfTI Resampling & Resizing Utility
-----------------------------------
• Reads every *.nii* / *.nii.gz* file in *input_dir*.
• Resamples to **1 × 1 × 1 mm** voxel spacing (linear interpolation).
• Resizes to **192 × 192 × 192** voxels (keeps origin & direction).
• Saves the processed image with the **same file‑name** inside *output_dir*.
• Prints old/new shape & spacing for quick verification.

Requirements
============
```
pip install SimpleITK
```

Edit `INPUT_DIR` / `OUTPUT_DIR` if you move the folders.
"""

from __future__ import annotations
import os
from pathlib import Path
from typing import Tuple, List
import SimpleITK as sitk
import matplotlib.pyplot as plt

# ────────────────────────────────────────────────────────────────────────────────
# CONFIGURATION (update if your folders change)
# ────────────────────────────────────────────────────────────────────────────────
INPUT_DIR  = Path("/Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/amos22_mri/imagesTr")
OUTPUT_DIR = Path("/Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/amos22_mri_resampled/imagesTr")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

TARGET_SPACING: Tuple[float, float, float] = (1.0, 1.0, 1.0)  # mm (x, y, z)
TARGET_SHAPE:   Tuple[int,  int,  int]   = (192, 192, 192)   # voxels (x, y, z)

# ────────────────────────────────────────────────────────────────────────────────
# HELPER FUNCTIONS
# ────────────────────────────────────────────────────────────────────────────────

def resample_to_spacing(img: sitk.Image, spacing: Tuple[float, float, float], is_label: bool = False) -> sitk.Image:
    """Resample *img* to *spacing*.
    Linear for images, NearestNeighbour for labels.
    """
    orig_spacing = img.GetSpacing()
    orig_size    = img.GetSize()
    new_size     = [int(round(osz * (osp / nsp))) for osz, osp, nsp in zip(orig_size, orig_spacing, spacing)]

    resampler = sitk.ResampleImageFilter()
    resampler.SetInterpolator(sitk.sitkNearestNeighbor if is_label else sitk.sitkLinear)
    resampler.SetOutputSpacing(spacing)
    resampler.SetSize(new_size)
    resampler.SetOutputDirection(img.GetDirection())
    resampler.SetOutputOrigin(img.GetOrigin())
    return resampler.Execute(img)


def resize_to_shape(img: sitk.Image, shape: Tuple[int, int, int], is_label: bool = False) -> sitk.Image:
    """Resize *img* to *shape* (changes spacing accordingly)."""
    curr_size    = img.GetSize()
    curr_spacing = img.GetSpacing()
    # new spacing so physical size stays similar after resizing
    new_spacing  = [csp * (csz / nsz) for csp, csz, nsz in zip(curr_spacing, curr_size, shape)]

    resampler = sitk.ResampleImageFilter()
    resampler.SetInterpolator(sitk.sitkNearestNeighbor if is_label else sitk.sitkLinear)
    resampler.SetOutputSpacing(new_spacing)
    resampler.SetSize(shape)
    resampler.SetOutputDirection(img.GetDirection())
    resampler.SetOutputOrigin(img.GetOrigin())
    return resampler.Execute(img)


# ────────────────────────────────────────────────────────────────────────────────
# PROCESS LOOP
# ────────────────────────────────────────────────────────────────────────────────
nii_files: List[Path] = sorted(INPUT_DIR.glob("*.nii*"))
if not nii_files:
    raise FileNotFoundError(f"No NIfTI files found in {INPUT_DIR}")

for idx, file_path in enumerate(nii_files, 1):
    img = sitk.ReadImage(str(file_path))
    orig_shape   = img.GetSize()
    orig_spacing = img.GetSpacing()

    # Step‑1  Resample to isotropic spacing -------------------------------------
    img_iso = resample_to_spacing(img, TARGET_SPACING)

    # Step‑2  Resize to target shape --------------------------------------------
    img_iso_resized = resize_to_shape(img_iso, TARGET_SHAPE)

    # Save & report -------------------------------------------------------------
    out_path = OUTPUT_DIR / file_path.name
    sitk.WriteImage(img_iso_resized, str(out_path))

    print(
        f"[{idx:03d}/{len(nii_files):03d}] {file_path.name}\n"
        f"  ‣ original shape  {orig_shape}  spacing {orig_spacing}\n"
        f"  ‣ new      shape  {img_iso_resized.GetSize()}  spacing {img_iso_resized.GetSpacing()}\n"
    )

print("\n✅ All volumes resampled & resized to 1 mm and 192³")


[001/040] amos_0507.nii.gz
  ‣ original shape  (320, 290, 72)  spacing (1.1875, 1.1875, 2.5)
  ‣ new      shape  (192, 192, 192)  spacing (1.9791666666666667, 1.7916666666666667, 0.9375)

[002/040] amos_0508.nii.gz
  ‣ original shape  (260, 80, 320)  spacing (1.40625, 1.5, 1.40625)
  ‣ new      shape  (192, 192, 192)  spacing (1.90625, 0.625, 2.34375)

[003/040] amos_0510.nii.gz
  ‣ original shape  (260, 104, 320)  spacing (1.40625, 1.5, 1.40625)
  ‣ new      shape  (192, 192, 192)  spacing (1.90625, 0.8125, 2.34375)

[004/040] amos_0514.nii.gz
  ‣ original shape  (260, 112, 320)  spacing (1.40625, 1.5, 1.40625)
  ‣ new      shape  (192, 192, 192)  spacing (1.90625, 0.875, 2.34375)

[005/040] amos_0517.nii.gz
  ‣ original shape  (400, 400, 115)  spacing (1.0, 1.0, 2.0)
  ‣ new      shape  (192, 192, 192)  spacing (2.0833333333333335, 2.0833333333333335, 1.1979166666666667)

[006/040] amos_0518.nii.gz
  ‣ original shape  (400, 120, 400)  spacing (1.0, 2.0, 1.0)
  ‣ new      shape  (192,

In [5]:
# Resample and reshape AMOS22 MRI dataset to 1x1x1 mm voxel size and target shape

# Import required libraries
import numpy as np
import os
import nibabel as nib
import matplotlib.pyplot as plt
from scipy.ndimage import zoom

# Define paths
input_dir = "/Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/amos22_mri/imagesTr/"
output_dir = "/Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/amos22_mri_resampled/imagesTr/"

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Define target voxel spacing and shape
target_spacing = [1.0, 1.0, 1.0]  # mm
target_shape = [192, 192, 192]

# Loop over all NIfTI files in the input directory
for filename in sorted(os.listdir(input_dir)):
    if filename.endswith(".nii") or filename.endswith(".nii.gz"):
        print(f"Processing: {filename}")
        
        # Load the NIfTI file
        img_path = os.path.join(input_dir, filename)
        img_nii = nib.load(img_path)
        img_data = img_nii.get_fdata()
        img_affine = img_nii.affine
        
        # Print some info
        print(f"  Original shape: {img_data.shape}")
        print(f"  Affine:\n{img_affine}")
        
        # Extract voxel dimensions from the affine matrix
        voxel_spacing = np.abs(img_affine[:3, :3]).sum(axis=0)
        print(f"  Original voxel spacing: {voxel_spacing}")

        # Compute the rescaling factors
        scale_factors = voxel_spacing / target_spacing
        print(f"  Scale factors: {scale_factors}")
        
        # Resample to isotropic spacing
        img_resampled = zoom(img_data, scale_factors, order=3, mode='nearest', prefilter=False)
        print(f"  Resampled shape: {img_resampled.shape}")
        
        # Compute resizing factors to target shape
        resize_factors = (
            target_shape[0] / img_resampled.shape[0],
            target_shape[1] / img_resampled.shape[1],
            target_shape[2] / img_resampled.shape[2]
        )
        print(f"  Resize factors: {resize_factors}")
        
        # Resize to target shape
        img_resized = zoom(img_resampled, resize_factors, order=3, mode='nearest', prefilter=False)
        print(f"  Final shape: {img_resized.shape}")
        
        # Save the processed image
        # Adjust the affine to reflect the new voxel spacing
        new_affine = np.copy(img_affine)
        new_affine[:3, :3] = np.diag(target_spacing)
        
        out_img = nib.Nifti1Image(img_resized.astype(np.float32), new_affine)
        out_path = os.path.join(output_dir, filename)
        nib.save(out_img, out_path)
        print(f"  Saved to: {out_path}\n")

print("Processing complete!")


Processing: amos_0507.nii.gz
  Original shape: (320, 290, 72)
  Affine:
[[   1.1875        0.            0.         -183.16921997]
 [   0.            1.1875        0.         -172.01393127]
 [   0.            0.            2.5         -60.35303497]
 [   0.            0.            0.            1.        ]]
  Original voxel spacing: [1.1875 1.1875 2.5   ]
  Scale factors: [1.1875 1.1875 2.5   ]
  Resampled shape: (380, 344, 180)
  Resize factors: (0.5052631578947369, 0.5581395348837209, 1.0666666666666667)
  Final shape: (192, 192, 192)
  Saved to: /Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/amos22_mri_resampled/imagesTr/amos_0507.nii.gz

Processing: amos_0508.nii.gz
  Original shape: (260, 80, 320)
  Affine:
[[ 1.40607679e+00  1.51430687e-03  2.20219977e-02 -2.10871536e+02]
 [-3.88847082e-04  1.49835980e+00 -6.57284036e-02 -6.30484047e+01]
 [-2.20642835e-02  7.00951442e-02  1.40454042e+00 -2.33261093e+02]
 [ 0.00000000e+00  0.00000000e+00  0.0000000

In [7]:
# Resample and reshape AMOS22 MRI dataset to 1x1x1 mm voxel size and target shape using SimpleITK

import os
import numpy as np
import SimpleITK as sitk
from pathlib import Path

# Define paths
input_dir = Path("/Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/amos22_mri/imagesTr/")
output_dir = Path("/Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/amos22_mri_resampled_itk/imagesTr/")
output_dir.mkdir(parents=True, exist_ok=True)

# Define target voxel spacing and shape
target_spacing = (1.0, 1.0, 1.0)  # mm (x, y, z)
target_shape = (192, 192, 192)    # voxels (x, y, z)

def resample_to_spacing(img, target_spacing, is_label=False):
    """Resample image to the target voxel spacing."""
    original_spacing = img.GetSpacing()
    original_size = img.GetSize()
    new_size = [
        int(round(osz * (osp / tsp)))
        for osz, osp, tsp in zip(original_size, original_spacing, target_spacing)
    ]
    
    resampler = sitk.ResampleImageFilter()
    resampler.SetInterpolator(sitk.sitkNearestNeighbor if is_label else sitk.sitkLinear)
    resampler.SetOutputSpacing(target_spacing)
    resampler.SetSize(new_size)
    resampler.SetOutputDirection(img.GetDirection())
    resampler.SetOutputOrigin(img.GetOrigin())
    resampler.SetDefaultPixelValue(img.GetPixelIDValue())
    
    img_resampled = resampler.Execute(img)
    return img_resampled

def resize_to_target_shape(img, target_shape, is_label=False):
    """Resize image to the target shape by adjusting spacing."""
    current_size = img.GetSize()
    current_spacing = img.GetSpacing()
    new_spacing = [
        csp * (csz / tsz)
        for csp, csz, tsz in zip(current_spacing, current_size, target_shape)
    ]
    
    resampler = sitk.ResampleImageFilter()
    resampler.SetInterpolator(sitk.sitkNearestNeighbor if is_label else sitk.sitkLinear)
    resampler.SetOutputSpacing(new_spacing)
    resampler.SetSize(target_shape)
    resampler.SetOutputDirection(img.GetDirection())
    resampler.SetOutputOrigin(img.GetOrigin())
    resampler.SetDefaultPixelValue(img.GetPixelIDValue())
    
    img_resized = resampler.Execute(img)
    return img_resized

# Loop over all NIfTI files in the input directory
for filename in sorted(input_dir.glob("*.nii*")):
    print(f"Processing: {filename.name}")
    
    # Load the NIfTI file using SimpleITK
    img = sitk.ReadImage(str(filename))
    
    # Print some info
    print(f"  Original size: {img.GetSize()}")
    print(f"  Original spacing: {img.GetSpacing()}")
    
    # Step 1: Resample to isotropic spacing
    img_resampled = resample_to_spacing(img, target_spacing)
    print(f"  Resampled size: {img_resampled.GetSize()}")
    print(f"  Resampled spacing: {img_resampled.GetSpacing()}")
    
    # Step 2: Resize to target shape
    img_resized = resize_to_target_shape(img_resampled, target_shape)
    print(f"  Resized size: {img_resized.GetSize()}")
    print(f"  Resized spacing: {img_resized.GetSpacing()}")
    
    # Save the processed image
    output_path = output_dir / filename.name
    sitk.WriteImage(img_resized, str(output_path))
    print(f"  Saved to: {output_path}\n")

print("✅ Processing complete!")


Processing: amos_0507.nii.gz
  Original size: (320, 290, 72)
  Original spacing: (1.1875, 1.1875, 2.5)
  Resampled size: (380, 344, 180)
  Resampled spacing: (1.0, 1.0, 1.0)
  Resized size: (192, 192, 192)
  Resized spacing: (1.9791666666666667, 1.7916666666666667, 0.9375)
  Saved to: /Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/amos22_mri_resampled_itk/imagesTr/amos_0507.nii.gz

Processing: amos_0508.nii.gz
  Original size: (260, 80, 320)
  Original spacing: (1.40625, 1.5, 1.40625)
  Resampled size: (366, 120, 450)
  Resampled spacing: (1.0, 1.0, 1.0)
  Resized size: (192, 192, 192)
  Resized spacing: (1.90625, 0.625, 2.34375)
  Saved to: /Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/amos22_mri_resampled_itk/imagesTr/amos_0508.nii.gz

Processing: amos_0510.nii.gz
  Original size: (260, 104, 320)
  Original spacing: (1.40625, 1.5, 1.40625)
  Resampled size: (366, 156, 450)
  Resampled spacing: (1.0, 1.0, 1.0)
  Resize

In [ ]:
import os
from pathlib import Path
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt

# Define the folders
original_dir = Path("/Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/amos22_mri/imagesTr/")
resampled_dir = Path("/Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/amos22_mri_resampled/imagesTr/")

# Find all NIfTI files in both directories
original_files = sorted(original_dir.glob("*.nii*"))
resampled_files = sorted(resampled_dir.glob("*.nii*"))

# Create a mapping from filename to resampled file
resampled_map = {f.name: f for f in resampled_files}

# Loop through each original file and compare with resampled
for original_file in original_files:
    filename = original_file.name
    if filename not in resampled_map:
        print(f"❌ Skipping {filename} (no matching resampled file)")
        continue
    
    resampled_file = resampled_map[filename]
    print(f"✅ Visualizing {filename}")
    
    # Load the images
    original_img = sitk.ReadImage(str(original_file))
    resampled_img = sitk.ReadImage(str(resampled_file))
    
    # Convert to numpy arrays
    original_np = sitk.GetArrayFromImage(original_img)   # (Z, Y, X)
    resampled_np = sitk.GetArrayFromImage(resampled_img)
    
    # Choose middle slice (Z-axis)
    original_mid_slice = original_np.shape[0] // 2
    resampled_mid_slice = resampled_np.shape[0] // 2
    
    # Plot side-by-side
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    axs[0].imshow(original_np[original_mid_slice, :, :], cmap='gray')
    axs[0].set_title(
        f"Original: {filename}\nShape: {original_np.shape}, Spacing: {original_img.GetSpacing()}"
    )
    axs[0].axis('off')
    
    axs[1].imshow(resampled_np[resampled_mid_slice, :, :], cmap='gray')
    axs[1].set_title(
        f"Resampled: {filename}\nShape: {resampled_np.shape}, Spacing: {resampled_img.GetSpacing()}"
    )
    axs[1].axis('off')
    
    plt.suptitle(f"Comparison: {filename}")
    plt.show()


AMOS Resample

In [23]:
import numpy as np
import os
import nibabel as nib
from scipy.ndimage import zoom

# Define paths
input_dir = "/Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/amos22_mri/imagesTs/"
output_dir = "/Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/amos22_mri_resampled_new/imagesTs/"

labels_dir = "/Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/amos22_mri/labelsTs/"
labels_out_dir = "/Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/amos22_mri_resampled_new/labelsTs/"

# Ensure output directories exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(labels_out_dir, exist_ok=True)

# Target voxel spacing and shape
target_spacing = [1.0, 1.0, 1.0]  # mm
target_shape = [192, 192, 192]

# Process each image
for filename in sorted(os.listdir(input_dir)):
    if filename.endswith(".nii") or filename.endswith(".nii.gz"):
        print(f"Processing: {filename}")

        # Load image
        img_path = os.path.join(input_dir, filename)
        img_nii = nib.load(img_path)
        img_data = img_nii.get_fdata()
        img_affine = img_nii.affine

        # Calculate voxel spacing from affine
        voxel_spacing = np.sqrt((img_affine[:3, :3] ** 2).sum(axis=0))
        print(f"  Original shape: {img_data.shape}")
        print(f"  Original voxel spacing: {voxel_spacing}")

        # Compute scale factors
        scale_factors = voxel_spacing / target_spacing
        print(f"  Scale factors: {scale_factors}")

        # Resample image to isotropic spacing
        img_resampled = zoom(img_data, scale_factors, order=3, mode='nearest', prefilter=False)
        print(f"  Resampled shape: {img_resampled.shape}")

        # Compute resize factors to target shape
        resize_factors = [
            target_shape[i] / img_resampled.shape[i] for i in range(3)
        ]
        print(f"  Resize factors: {resize_factors}")

        # Resize to target shape
        img_resized = zoom(img_resampled, resize_factors, order=3, mode='nearest', prefilter=False)
        print(f"  Final image shape: {img_resized.shape}")

        # Save resampled image
        new_affine = np.copy(img_affine)
        new_affine[:3, :3] = np.diag(target_spacing)
        out_img = nib.Nifti1Image(img_resized.astype(np.float32), new_affine)
        out_path = os.path.join(output_dir, filename)
        nib.save(out_img, out_path)
        print(f"  Saved image to: {out_path}")

        # Process corresponding label
        label_path = os.path.join(labels_dir, filename)
        if os.path.exists(label_path):
            print(f"  Processing label: {filename}")
            label_nii = nib.load(label_path)
            label_data = label_nii.get_fdata()

            # Resample label with nearest neighbor interpolation
            label_resampled = zoom(label_data, scale_factors, order=0, mode='nearest', prefilter=False)
            print(f"  Label resampled shape: {label_resampled.shape}")

            # Resize label to target shape
            label_resized = zoom(label_resampled, resize_factors, order=0, mode='nearest', prefilter=False)
            print(f"  Final label shape: {label_resized.shape}")

            # Save resampled label
            label_out_img = nib.Nifti1Image(label_resized.astype(np.uint8), new_affine)
            label_out_path = os.path.join(labels_out_dir, filename)
            nib.save(label_out_img, label_out_path)
            print(f"  Saved label to: {label_out_path}")
        else:
            print(f"  ⚠️ Label file not found for {filename} — skipping label processing.")

        print()

print("✅ All images and labels have been processed.")


Processing: amos_0500.nii.gz
  Original shape: (512, 512, 246)
  Original voxel spacing: [0.67578125 0.67578125 2.        ]
  Scale factors: [0.67578125 0.67578125 2.        ]
  Resampled shape: (346, 346, 492)
  Resize factors: [0.5549132947976878, 0.5549132947976878, 0.3902439024390244]
  Final image shape: (192, 192, 192)
  Saved image to: /Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/amos22_mri_resampled_new/imagesTs/amos_0500.nii.gz
  ⚠️ Label file not found for amos_0500.nii.gz — skipping label processing.

Processing: amos_0501.nii.gz
  Original shape: (1024, 1024, 48)
  Original voxel spacing: [0.39062501 0.39062499 5.00000268]
  Scale factors: [0.39062501 0.39062499 5.00000268]
  Resampled shape: (400, 400, 240)
  Resize factors: [0.48, 0.48, 0.8]
  Final image shape: (192, 192, 192)
  Saved image to: /Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/amos22_mri_resampled_new/imagesTs/amos_0501.nii.gz
  ⚠️ Label fi

CHAOS Resample

In [2]:
import numpy as np
import os
import nibabel as nib
from scipy.ndimage import zoom

# Define paths
input_dir = "/Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/chaos/NIfTI_new/MR/images/"
output_dir = "/Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/resampled/train/chaos/images/"

labels_dir = "/Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/chaos/NIfTI_new/MR/labels/"
labels_out_dir = "/Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/resampled/train/chaos/labels/"

# Ensure output directories exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(labels_out_dir, exist_ok=True)

# Target voxel spacing and shape
target_spacing = [1.0, 1.0, 1.0]  # mm
target_shape = [192, 192, 192]

# Process each image
for filename in sorted(os.listdir(input_dir)):
    if filename.endswith(".nii") or filename.endswith(".nii.gz"):
        print(f"Processing: {filename}")

        # Load image
        img_path = os.path.join(input_dir, filename)
        img_nii = nib.load(img_path)
        img_data = img_nii.get_fdata()
        img_affine = img_nii.affine

        # Calculate voxel spacing from affine
        voxel_spacing = np.sqrt((img_affine[:3, :3] ** 2).sum(axis=0))
        print(f"  Original shape: {img_data.shape}")
        print(f"  Original voxel spacing: {voxel_spacing}")

        # Compute scale factors
        scale_factors = voxel_spacing / target_spacing
        print(f"  Scale factors: {scale_factors}")

        # Resample image to isotropic spacing
        img_resampled = zoom(img_data, scale_factors, order=3, mode='nearest', prefilter=False)
        print(f"  Resampled shape: {img_resampled.shape}")

        # Compute resize factors to target shape
        resize_factors = [
            target_shape[i] / img_resampled.shape[i] for i in range(3)
        ]
        print(f"  Resize factors: {resize_factors}")

        # Resize to target shape
        img_resized = zoom(img_resampled, resize_factors, order=3, mode='nearest', prefilter=False)
        print(f"  Final image shape: {img_resized.shape}")

        # Save resampled image
        new_affine = np.copy(img_affine)
        new_affine[:3, :3] = np.diag(target_spacing)
        out_img = nib.Nifti1Image(img_resized.astype(np.float32), new_affine)
        out_path = os.path.join(output_dir, filename)
        nib.save(out_img, out_path)
        print(f"  Saved image to: {out_path}")

        # Process corresponding label
        label_path = os.path.join(labels_dir, filename)
        if os.path.exists(label_path):
            print(f"  Processing label: {filename}")
            label_nii = nib.load(label_path)
            label_data = label_nii.get_fdata()

            # Resample label with nearest neighbor interpolation
            label_resampled = zoom(label_data, scale_factors, order=0, mode='nearest', prefilter=False)
            print(f"  Label resampled shape: {label_resampled.shape}")

            # Resize label to target shape
            label_resized = zoom(label_resampled, resize_factors, order=0, mode='nearest', prefilter=False)
            print(f"  Final label shape: {label_resized.shape}")

            # Save resampled label
            label_out_img = nib.Nifti1Image(label_resized.astype(np.uint8), new_affine)
            label_out_path = os.path.join(labels_out_dir, filename)
            nib.save(label_out_img, label_out_path)
            print(f"  Saved label to: {label_out_path}")
        else:
            print(f"  ⚠️ Label file not found for {filename} — skipping label processing.")

        print()

print("✅ All images and labels have been processed.")


Processing: Patient_10_T1_IN.nii.gz
  Original shape: (256, 256, 50)
  Original voxel spacing: [1.89453125 1.89453125 5.5       ]
  Scale factors: [1.89453125 1.89453125 5.5       ]
  Resampled shape: (485, 485, 275)
  Resize factors: [0.3958762886597938, 0.3958762886597938, 0.6981818181818182]
  Final image shape: (192, 192, 192)
  Saved image to: /Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/resampled/train/chaos/images/Patient_10_T1_IN.nii.gz
  Processing label: Patient_10_T1_IN.nii.gz
  Label resampled shape: (485, 485, 275)
  Final label shape: (192, 192, 192)
  Saved label to: /Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/resampled/train/chaos/labels/Patient_10_T1_IN.nii.gz

Processing: Patient_10_T2.nii.gz
  Original shape: (256, 256, 36)
  Original voxel spacing: [1.69921875 1.69921875 7.69999981]
  Scale factors: [1.69921875 1.69921875 7.69999981]
  Resampled shape: (435, 435, 277)
  Resize factors: [0.4413793

**CT AMOS Resample**

In [1]:
import numpy as np
import os
import nibabel as nib
from scipy.ndimage import zoom

# Define paths (update these to your actual CT data locations)
input_dir = "/Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/amos22_ct/images"
output_dir = "/Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/resampled/train/amos_ct/images"

labels_dir = "/Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/amos22_ct/labels"
labels_out_dir = "/Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/resampled/train/amos_ct/labels"

# Ensure output directories exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(labels_out_dir, exist_ok=True)

# Target voxel spacing and shape
target_spacing = [1.0, 1.0, 1.0]  # mm
target_shape = [192, 192, 192]

def preprocess_ct(image):
    """Preprocess CT image with appropriate window settings."""
    window_min, window_max = -160, 240  # Typical abdominal window
    image = np.clip(image, window_min, window_max)
    image = (image - window_min) / (window_max - window_min)
    return image

# Process each image
for filename in sorted(os.listdir(input_dir)):
    if filename.endswith(".nii") or filename.endswith(".nii.gz"):
        print(f"Processing: {filename}")

        # Load image
        img_path = os.path.join(input_dir, filename)
        img_nii = nib.load(img_path)
        img_data = img_nii.get_fdata()
        img_affine = img_nii.affine

        # Calculate voxel spacing from affine
        voxel_spacing = np.sqrt((img_affine[:3, :3] ** 2).sum(axis=0))
        print(f"  Original shape: {img_data.shape}")
        print(f"  Original voxel spacing: {voxel_spacing}")

        # Compute scale factors
        scale_factors = voxel_spacing / target_spacing
        print(f"  Scale factors: {scale_factors}")

        # Resample image to isotropic spacing
        img_resampled = zoom(img_data, scale_factors, order=3, mode='nearest', prefilter=False)
        print(f"  Resampled shape: {img_resampled.shape}")

        # Compute resize factors to target shape
        resize_factors = [
            target_shape[i] / img_resampled.shape[i] for i in range(3)
        ]
        print(f"  Resize factors: {resize_factors}")

        # Resize to target shape
        img_resized = zoom(img_resampled, resize_factors, order=3, mode='nearest', prefilter=False)
        print(f"  Final image shape: {img_resized.shape}")

        # --- CT-specific preprocessing ---
        # img_resized = preprocess_ct(img_resized)

        # Save resampled image
        new_affine = np.copy(img_affine)
        new_affine[:3, :3] = np.diag(target_spacing)
        out_img = nib.Nifti1Image(img_resized.astype(np.float32), new_affine)
        out_path = os.path.join(output_dir, filename)
        nib.save(out_img, out_path)
        print(f"  Saved image to: {out_path}")

        # Process corresponding label
        label_path = os.path.join(labels_dir, filename)
        if os.path.exists(label_path):
            print(f"  Processing label: {filename}")
            label_nii = nib.load(label_path)
            label_data = label_nii.get_fdata()

            # Resample label with nearest neighbor interpolation
            label_resampled = zoom(label_data, scale_factors, order=0, mode='nearest', prefilter=False)
            print(f"  Label resampled shape: {label_resampled.shape}")

            # Resize label to target shape
            label_resized = zoom(label_resampled, resize_factors, order=0, mode='nearest', prefilter=False)
            print(f"  Final label shape: {label_resized.shape}")

            # Save resampled label
            label_out_img = nib.Nifti1Image(label_resized.astype(np.uint8), new_affine)
            label_out_path = os.path.join(labels_out_dir, filename)
            nib.save(label_out_img, label_out_path)
            print(f"  Saved label to: {label_out_path}")
        else:
            print(f"  ⚠️ Label file not found for {filename} — skipping label processing.")

        print()

print("✅ All CT images and labels have been processed.")

Processing: amos_0001.nii.gz
  Original shape: (768, 768, 90)
  Original voxel spacing: [0.5703125 0.5703125 5.       ]
  Scale factors: [0.5703125 0.5703125 5.       ]
  Resampled shape: (438, 438, 450)
  Resize factors: [0.4383561643835616, 0.4383561643835616, 0.4266666666666667]
  Final image shape: (192, 192, 192)
  Saved image to: /Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/resampled/train/amos_ct/images/amos_0001.nii.gz
  Processing label: amos_0001.nii.gz
  Label resampled shape: (438, 438, 450)
  Final label shape: (192, 192, 192)
  Saved label to: /Users/fransiskusbudi/UoE/Dissertation/multimodal_segmentation_project/datasets/resampled/train/amos_ct/labels/amos_0001.nii.gz

Processing: amos_0004.nii.gz
  Original shape: (512, 512, 78)
  Original voxel spacing: [0.78200001 0.78200001 5.        ]
  Scale factors: [0.78200001 0.78200001 5.        ]
  Resampled shape: (400, 400, 390)
  Resize factors: [0.48, 0.48, 0.49230769230769234]
  Final im